# 베드락을 이용하여 리뷰, 상담내역 요약 및 분석 요약하기

> *이 노트북은 SageMaker Studio*의 `JupyterLab` 에서 테스트하였습니다. 

## 소개

이 노트북에서는 리테일 고객이 크기가 큰 문서를 요약하는 방법을 보여드리겠습니다.대용량 문서로 작업할 때 입력 텍스트가 모델 컨텍스트 길이에 맞지 않거나, 모델이 대용량 문서를 인식하지 못하거나, 메모리 부족 오류 등으로 인해 몇 가지 문제에 직면할 수 있습니다. 이러한 문제를 해결하기 위해 청킹 및 연쇄 프롬프트 개념을 기반으로 하는 아키텍처를 보여드리겠습니다. 이 아키텍처는 언어 모델로 구동되는 애플리케이션을 개발하는 데 널리 사용되는 프레임워크인 LangChain을 활용합니다.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html)은 언어 모델로 구동되는 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크의 핵심 측면을 통해 다양한 구성 요소를 연결하여 고급 사용 사례를 만들어 대규모 언어 모델을 보강할 수 있습니다.


#### 컨텍스트

이 노트북에서는 LangChain 프레임워크 내에서 Amazon Bedrock과 통합하여 사용는 방법과 PromptTemplate의 도움으로 텍스트를 생성하는 데 어떻게 사용될 수 있는지 살펴보겠습니다.


#### 사용사례

이 접근 방식은 통화 녹취록, 회의 녹취록, 책, 기사, 블로그 게시물 및 상품/서비스 관련 콘텐츠를 요약하는 데 사용할 수 있습니다.


#### 구현 방법

이 사용 사례를 보여주기 위해 이 노트북에서는 고객의 이전 제품 설명을 기반으로 신규 제품 설명을 생성하는 방법을 보여드리며, Boto3 클라이언트와 함께 Amazon Bedrock API를 사용하는 Anthropic Claude 모델을 사용하겠습니다.


## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) 베드락에 연결해야 합니다.

우선 사전에 설치가 필요한 패키지들을 설치하세요. 그 이후에 셋업에 필요한 라이브러리들을 설치합니다. 

#### 앞 부분은 이전 실습 과정에서 했던 내용과 동일합니다

In [ ]:
!pip install -q -r dependencies/requirements.txt
!pip install Unstructured -q

In [ ]:
import json
import os
import sys
import boto3
import botocore
from botocore.config import Config
from langchain_community.chat_models import BedrockChat
from langchain.chains.summarize import load_summarize_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# this is setting the maximum number of times boto3 will attempt our call to bedrock
my_region = "us-west-2" # change this value to point to a different region
my_config = Config(
    region_name = my_region,
    signature_version = 'v4',
    retries = {
        'max_attempts': 3,
        'mode': 'standard'
    }
)

# this creates our client we will use to access Bedrock
bedrock_rt = boto3.client("bedrock-runtime", config = my_config)
bedrock = boto3.client("bedrock")

## 베드락 LLM 모델 호출하기

LLM에서 Bedrock 클래스의 인스턴스를 생성하는 것으로 시작하겠습니다. 여기에는 Amazon Bedrock에서 사용할 수 있는 모델의 ARN인 model_id가 필요합니다.

선택적으로 이전에 생성한 boto3 클라이언트를 전달할 수 있으며, `temperature`, `top_p`, `max_tokens` 또는 `stop_sequences`와 같은 매개 변수를 보유할 수 있는 일부 `model_kwargs`도 전달할 수 있습니다(매개 변수에 대한 자세한 내용은 Amazon Bedrock 콘솔에서 살펴볼 수 있습니다).

Amazon Bedrock에서 사용 가능한 텍스트 생성 모델 ID에 대한 [설명서](https://docs.aws.amazon.com/ko_kr/bedrock/latest/userguide/model-ids-arns.html)를 확인하세요.

모델마다 지원하는 `model_kwargs`가 다르다는 점에 유의하세요.

In [ ]:
sonnet_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
# to switch to claude v3 Haiku you can use this id
haiku_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

model_kwargs =  { 
    "max_tokens": 8192,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["Human"],
}

# this defines a sonnet object
llm = BedrockChat(
    client=bedrock_rt,
    model_id=sonnet_model_id,
    model_kwargs=model_kwargs,
)

# for any of our chains we can change out the model by simply substituting the sonnet model object with this haiku object.
haiku_model = BedrockChat(
    client=bedrock_rt,
    model_id=haiku_model_id,
    model_kwargs=model_kwargs,
)

## 리뷰 텍스트 요약하기

#### 여기서는 온라인몰의 리뷰 샘플 100개를 활용해 실습합니다. AI assistant를 활용하면 사람보다 훨씬 빠른 속도로 리뷰를 요약하고 분석할 수 있습니다.

In [ ]:
import xml.etree.ElementTree as ET

def extract_reviews_to_string(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    reviews_string = ET.tostring(root, encoding='unicode')
    return reviews_string

file_path = 'reviews.xml'
reviews_content = extract_reviews_to_string(file_path)
print(reviews_content[:1000])  

In [ ]:
# Invoke Example, request is a variable we must fill in when we invoke our chain
prompt_template = """
Human: 
You are an assistant who reads cosmetic reviews from users and analyzes them according to requests. Read the review below and answer the question:

{reviews}

Question: {input}
Assistant:"""

# pass our list of messages to our ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(prompt_template)

# use | to chain together multiple components
chain = prompt | llm 

In [ ]:
question = "이 제품에 대한 전반적인 만족도가 어때? 장단점을 bullet point 형식으로 요약해줘"

In [ ]:
response = chain.invoke({"input": question, "reviews": reviews_content})
print(response.content)

In [ ]:
question = "리뷰에서 가장 많이 등장한 용어가 뭔지 5개를 선정해서 설명해줘"
response = chain.invoke({"input": question, "reviews": reviews_content})

In [ ]:
print(response.content)

## Summarizing chunks and combining them

다른 문서에서 토큰의 수가 일정하다고 가정하면 문제가 없을 것입니다. LangChain의 load_summarize_chain을 사용하여 텍스트를 요약해 보겠습니다. load_summarize_chain은 stuff, map_reduce, refine의 세 가지 요약 방법을 제공합니다.

* stuff는 모든 청크를 하나의 프롬프트에 넣습니다. 따라서 토큰의 최대 한도에 도달하게 됩니다.
* map_reduce는 각 청크를 요약하고, 요약을 결합한 다음, 결합된 요약을 요약합니다. 결합된 요약이 너무 크면 오류가 발생할 수 있습니다.
* refine은 첫 번째 청크를 요약한 다음 첫 번째 요약으로 두 번째 청크를 요약합니다. 모든 청크가 요약될 때까지 동일한 프로세스가 반복됩니다.

map_reduce와 refine은 LLM을 여러 번 호출하므로 최종 요약을 얻는 데 시간이 걸립니다. 여기에서는 stuff는를 사용해 보겠습니다.

'map_reduce' 체인은 큰 문서를 관리하기 쉬운 작은 덩어리로 분할하여 문서 처리를 처리하도록 설계되었습니다. 이 체인은 각 조각에 초기 프롬프트를 사용하여 문서의 특정 섹션을 기반으로 요약 또는 답변을 생성합니다. 또한 MapReduceDocumentsChain은 생성된 출력을 가져와 다른 프롬프트를 사용하여 결합하여 전체 문서에 대한 포괄적이고 일관된 요약 또는 답변을 생성합니다. load_summarize_chain 함수를 사용하여 'map_reduce' 체인을 설정하고 출력 요약을 얻습니다.

In [ ]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredXMLLoader

In [ ]:
loader = UnstructuredXMLLoader(
    "./reviews.xml"
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " "], chunk_size=4000, chunk_overlap=100 
)
docs = text_splitter.split_documents(documents=documents)
docs[0]

In [ ]:
from langchain.prompts import PromptTemplate
import textwrap

template = """
You are an assistant tasked with summarizing cosmetic reviews. 
Your goal is to analyze the text provided and generate a concise summary that clearly identifies the key points of satisfaction and dissatisfaction expressed in the reviews. 
Please format your response with appropriate line breaks to enhance readability, making the summary easy to follow and visually appealing.

<reviews>
{text}
</reviews>

Please provide a concise summary with well-organized line breaks that captures the main points of satisfaction and dissatisfaction based on the review text provided. 
Make sure the summary is structured and each point is clearly to facilitate easy reading in Korean.

Assistant:
"""
prompt = PromptTemplate.from_template(template)

chain = load_summarize_chain(llm, prompt=prompt, chain_type="stuff")
output_summary = chain.invoke({"input_documents": docs})

In [ ]:
print(output_summary["output_text"])

## Streamlit 어플리케이션 수행하기

In [ ]:
%%writefile ../demo-app.py

from langchain.prompts import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
import streamlit as st
import xml.etree.ElementTree as ET

def extract_reviews_to_string(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        reviews_string = ET.tostring(root, encoding='unicode')
        return reviews_string
    except Exception as e:
        return str(e)

def get_review_by_number(reviews, number):
    try:
        root = ET.fromstring(reviews)
        review_element = root.findall('review')[number - 1]
        review_text = ''.join(review_element.itertext())
        rating_prefix = "rating: "
        contents_prefix = ", contents: "
        rating_start = review_text.find(rating_prefix) + len(rating_prefix)
        contents_start = review_text.find(contents_prefix) + len(contents_prefix)
        rating = int(review_text[rating_start:review_text.find(',', rating_start)])
        contents = review_text[contents_start:].strip()
        star_rating = '⭐' * rating
        formatted_output = f"평점 : {star_rating}\n\n{contents}"
        return formatted_output
    except IndexError:
        return "Review number out of range" 
    except Exception as e:
        return str(e)

st.set_page_config(layout="wide", page_title="텍스트 요약")
st.title("리뷰 요약 및 고객 감성 분석")

reviews_content = extract_reviews_to_string('./PromptEngineering/reviews.xml')
model_kwargs =  { "max_tokens": 8192, "temperature": 0}
llm = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs=model_kwargs) 

# 분석용 체인 생성
analysis_template = """
Human: 
You are an assistant tasked with reading cosmetic reviews from users and analyzing them according to specific requests. 
Please ensure your responses are neatly organized and visually appealing. 
Summarize the overall analysis results and highlight key phrases or keywords that customers have expressed the most satisfaction or dissatisfaction with.

Read the review below and answer the question:
{review}

Question: {input}
Assistant:"""

analysis_prompt = ChatPromptTemplate.from_template(analysis_template)
analysis_chain = analysis_prompt | llm 

# 자동 답변용 체인 생성
reply_template = """
Human:
You are an assistant who reads selected cosmetic reviews and generates responses from the perspective of the seller. 
Your task is to respond as warmly and kindly as possible, ensuring your responses are concise and accurately reflect the content of the user's question. 
When the customer’s satisfaction appears to be less than ideal, emphasize a sincere apology and offer solutions or explanations to address their concerns.
Enhance your responses with appropriate emojis that match the tone or content of the review to make your replies more engaging and relatable.
<review>
{input}
</review>

Assistant:"""

reply_prompt = ChatPromptTemplate.from_template(reply_template)
reply_chain = reply_prompt | llm 

# 분석 기능
default_question = "사용자들이 이 제품에 대해 느끼는 장점과 단점을 정리해주세요"
question = st.text_input('제품 리뷰에 대해 궁금한 점을 알려주세요:', value=default_question)
if st.button('Analyze'):
    response = analysis_chain.invoke({"input": question, "review": reviews_content})
    st.write(response.content)

# 답변 기능
review_number = st.number_input('리뷰 ID를 입력해주세요 (1-100 / 자세한 긍정 댓글 = 32 / 부정 댓글 = 2 (가상 생성됨)):', value=1, min_value=1, max_value=100, step=1)
if st.button('댓글 보기'):
    review_output = get_review_by_number(reviews_content, int(review_number))
    st.session_state['review_output'] = review_output  # Save output to session state
    st.write(review_output)
if st.button('답글 자동 생성'):
    if 'review_output' in st.session_state and st.session_state['review_output']:
        response = reply_chain.invoke({"input": st.session_state['review_output']})
        st.write(response.content)
    else:
        st.error("Please load a review first.")

### 앞서 실행했던 Streamlit 페이지를 확인해보세요